In [ ]:
!pip install pandas requests google-auth-oauthlib google-auth-httplib2 google-api-python-client openai PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests
import io
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from datetime import datetime, timedelta
import PyPDF2
from PyPDF2 import PdfReader
from openai import OpenAI
import re


In [ ]:
# # Authenticate and create Google Drive API client
# auth.authenticate_user()
# creds, _ = default()
# drive_service = build('drive', 'v3', credentials=creds)

In [ ]:
# Set up OpenAI client
client = OpenAI(api_key='No leaks!')

In [ ]:
def extract_pdf_text(pdf_url):
    try:
        # Check if it's a Google Drive link
        if 'drive.google.com' in pdf_url:
            # Extract the file ID
            file_id = re.findall(r'id=([\w-]+)', pdf_url)
            if not file_id:
                file_id = re.findall(r'/d/([\w-]+)', pdf_url)
            if not file_id:
                raise ValueError("Could not extract Google Drive file ID")

            file_id = file_id[0]
            # Construct the direct download link
            pdf_url = f'https://drive.google.com/uc?export=download&id={file_id}'

        # Download the PDF
        response = requests.get(pdf_url)
        response.raise_for_status()  # Raise an exception for bad responses

        # Create a PDF reader object
        pdf_file = io.BytesIO(response.content)
        pdf_reader = PdfReader(pdf_file)

        # Extract text from all pages
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()

        return text
    except Exception as e:
        print(f"Error extracting text from PDF {pdf_url}: {e}")
        return None

In [ ]:
def analyze_text_with_gpt(text, prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{prompt}\n\nText: {text}"}
        ]
    )
    return response.choices[0].message.content

In [ ]:
def get_cyber_relevance(major):
    prompt = """
    Provide a relevance score from 0 to 1 indicating how closely the major is related to Cybersecurity, Computer Science, or Infomration Technology.
    Only return a relevance score, nothing else.
    """
    relevance = analyze_text_with_gpt(major, prompt)
    return relevance

In [ ]:
def parse_date(date_string):
    # Remove any leading/trailing whitespace
    date_string = date_string.strip()

    # Try parsing with different formats
    for fmt in ('%Y-%m-%d', '%Y/%m/%d', '%d-%m-%Y', '%d/%m/%Y', '%m-%d-%Y', '%m/%d/%Y'):
        try:
            return datetime.strptime(date_string, fmt)
        except ValueError:
            pass

    # If all else fails, try to extract date using regex
    date_pattern = r'\b\d{4}[-/]\d{1,2}[-/]\d{1,2}\b|\b\d{1,2}[-/]\d{1,2}[-/]\d{4}\b'
    match = re.search(date_pattern, date_string)
    if match:
        date_str = match.group(0)
        for fmt in ('%Y-%m-%d', '%Y/%m/%d', '%d-%m-%Y', '%d/%m/%Y', '%m-%d-%Y', '%m/%d/%Y'):
            try:
                return datetime.strptime(date_str, fmt)
            except ValueError:
                pass

    raise ValueError(f"Unable to parse date string: {date_string}")

In [ ]:
def calculate_time_since_graduation(grad_date):
    today = datetime.now()
    time_since_grad = today - grad_date
    years = time_since_grad.days // 365
    months = (time_since_grad.days % 365) // 30
    return f"{years} years and {months} months"

In [ ]:
def main():
    # Access the public Google Sheet
    sheet_id = 'NO LEAKS!'
    sheet_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx'
    response = requests.get(sheet_url)
    df = pd.read_excel(io.BytesIO(response.content))
    qualified_applicants = []
    not_qualified_applicants = []

    for _, row in df.iterrows():
        try:
            #transcript_url = row['Upload your electronic transcript with selectable text']
            #certificate_url = row['Upload your electronic graduation certificate with selectable text']
            #transcript_text = extract_pdf_text(transcript_url)
            #certificate_text = extract_pdf_text(certificate_url)

            # if transcript_text is None or certificate_text is None:
            #     print(f"Skipping applicant {row['National ID Number']} due to PDF extraction error")
            #     continue

            gpa, gpa_scale = float(row["GPA"]), int(row["GPA Average (Out of 4 or 5)"])
            #print(type(gpa), type(gpa_scale))
            cyber_relevance = float(get_cyber_relevance(row["Major"]))
            #print(cyber_relevance)
            grad_date = row['Graduation Date']
            time_since_graduation = calculate_time_since_graduation(grad_date)
            #print(time_since_graduation)
            english_score = float(row['English proficiency test result'])
            if row['English Proficiency Exam'] == 'IELTS':
                score_limiter = 6.5
            elif row['English Proficiency Exam'] == 'TOEFL':
                score_limiter = 79
            else:
                score_limiter = 105
            # Check qualifications
            if (cyber_relevance >= 0.7 and
                ((gpa_scale == 4 and gpa >= 3.2) or (gpa_scale == 5 and gpa >= 4)) and
                english_score >= score_limiter and
                (datetime.now() - grad_date) <= timedelta(days=5*365)):

                qualified_applicants.append({
                    'ID': row['National ID Number'],
                    'Name': row['Full Name(As your national ID)'],
                    'GPA': f"{gpa}/{gpa_scale}",
                    'Major': row["Major"],
                    'Cybersecurity Relevance': f"{cyber_relevance:.2f}",
                    'Graduation Date': grad_date.strftime('%Y-%m-%d'),
                    'Time Since Graduation': time_since_graduation,
                    'English Score': english_score,
                    'English exam': row['English Proficiency Exam']

                })
            else:
                not_qualified_applicants.append({
                    'ID': row['National ID Number'],
                    'Name': row['Full Name(As your national ID)'],
                    'GPA': f"{gpa}/{gpa_scale}",
                    'Major': row["Major"],
                    'Cybersecurity Relevance': f"{cyber_relevance:.2f}",
                    'Graduation Date': grad_date.strftime('%Y-%m-%d'),
                    'Time Since Graduation': time_since_graduation,
                    'English Score': english_score,
                    'English exam': row['English Proficiency Exam']
                })
        except Exception as e:
            print(f"Error processing applicant {row['National ID Number']}: {e}")
            continue

    print("Qualified Applicants:")
    for applicant in qualified_applicants:
        print(f"ID: {applicant['ID']}, Name: {applicant['Name']}, GPA: {applicant['GPA']}, "
              f"Major: {applicant['Major']}, Cybersecurity Relevance: {applicant['Cybersecurity Relevance']}, "
              f"Graduation Date: {applicant['Graduation Date']}, Time Since Graduation: {applicant['Time Since Graduation']}, English Score {applicant['English Score']}, English Exam {applicant['English exam']}")

    print("Not Qualified Applicants:")
    for applicant in not_qualified_applicants:
        print(f"ID: {applicant['ID']}, Name: {applicant['Name']}, GPA: {applicant['GPA']}, "
              f"Major: {applicant['Major']}, Cybersecurity Relevance: {applicant['Cybersecurity Relevance']}, "
              f"Graduation Date: {applicant['Graduation Date']}, Time Since Graduation: {applicant['Time Since Graduation']},  English Score {applicant['English Score']}, English Exam {applicant['English exam']}")

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
# cybersecurity_relevant_majors = [
#     "Cybersecurity",
#     "Computer Science",
#     "Information Technology",
#     "Network Security",
#     "Information Security",
#     "Software Engineering",
#     "Computer Engineering",
#     "Data Science",
#     "Cryptography",
#     "Digital Forensics",
#     "Cloud Computing",
#     "Network Administration",
#     "Systems Engineering",
#     "Security Management",
#     "Information Systems",
#     "Penetration Testing",
#     "Ethical Hacking",
#     "Artificial Intelligence",
#     "Risk Management",
#     "Incident Response"
#     ''
# ]


# for item in cybersecurity_relevant_majors:
#     print(item, get_cyber_relevance(item))

In [ ]:
print(get_cyber_relevance("Chemistry"))

0.1
